<a href="https://colab.research.google.com/github/Nicolenki7/Customer-Sentiment-Analysis-Product-Improvement-Drivers/blob/main/Sentiment_Analysis_and_Satisfaction_Drivers_in_E_commerce_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# CELDAS DE INSTALACIÓN Y DESCARGA DE RECURSOS

# Importar nltk para descargar los recursos necesarios
import nltk
import ssl

# Manejar el error de certificación común en algunos entornos
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Descargar los recursos clave:
nltk.download('stopwords')
nltk.download('vader_lexicon')

print("--- Configuración de NLTK en Colab completada con éxito. ---")

--- Configuración de NLTK en Colab completada con éxito. ---


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
# SCRIPT DE ANÁLISIS DE SENTIMIENTO CORREGIDO

import pandas as pd
import numpy as np
import re
# Ya no necesitamos re-importar NLTK, pero mantenemos las librerías
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import gc

# --- 1. Carga Inteligente (Simulación) ---

# Usamos el DataFrame de ejemplo para simular la carga exitosa
data = {
    'Id': [1, 2, 3, 4, 5, 6, 7],
    'ProductId': ['B001', 'B008', 'B006', 'B007', 'B007', 'B001', 'B008'],
    'Score': [5, 1, 5, 2, 4, 1, 5],
    'Text': [
        "I love this product, it tastes amazing and the delivery was fast.",
        "The package was damaged and the food was stale. Very disappointed.",
        "Excellent product! A bit pricey, but worth every penny.",
        "Taste is average, nothing special. Won't buy again.",
        "Good value for money. Will recommend it to friends.",
        "Absolutely horrible. The worst quality I have ever seen.",
        "Great taste and super fast shipping! Highly recommend this item."
    ]
}
df_reviews = pd.DataFrame(data)

# --- 2. Preprocesamiento de Texto Básico ---

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    # Se eliminan las Stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

df_reviews['Clean_Text'] = df_reviews['Text'].apply(clean_text)

# --- 3. Aplicación del Análisis de Sentimiento (VADER) ---

# Inicializar VADER (ahora que el recurso está descargado)
vader = SentimentIntensityAnalyzer()

# Aplicar VADER
df_reviews['Compound_Score'] = df_reviews['Clean_Text'].apply(lambda x: vader.polarity_scores(x)['compound'])

# --- 4. Limpieza de Memoria ---
del data
gc.collect()

print("\n--- ¡Éxito! Pre-Procesamiento y Sentimiento Finalizado ---")
print(df_reviews[['Id', 'Score', 'Text', 'Compound_Score']].head(7))


--- ¡Éxito! Pre-Procesamiento y Sentimiento Finalizado ---
   Id  Score                                               Text  \
0   1      5  I love this product, it tastes amazing and the...   
1   2      1  The package was damaged and the food was stale...   
2   3      5  Excellent product! A bit pricey, but worth eve...   
3   4      2  Taste is average, nothing special. Won't buy a...   
4   5      4  Good value for money. Will recommend it to fri...   
5   6      1  Absolutely horrible. The worst quality I have ...   
6   7      5  Great taste and super fast shipping! Highly re...   

   Compound_Score  
0          0.8402  
1         -0.7184  
2          0.6808  
3         -0.3089  
4          0.8720  
5         -0.8470  
6          0.8955  


In [4]:
import sqlite3
import pandas as pd
# Asumimos que df_reviews ya está en memoria con la columna 'Compound_Score'
#
# 1. Conexión a la Base de Datos SQLite (Temporal)
# Usaremos ':memory:' para una base de datos en RAM, muy rápido para portafolios.
conn = sqlite3.connect(':memory:')

# 2. Volcar el DataFrame a una Tabla SQL
# Insertamos solo las columnas necesarias para el análisis final
df_reviews[['Id', 'ProductId', 'Score', 'Compound_Score']].to_sql(
    'reviews_sentiment',    # Nombre de la tabla
    conn,                   # Conexión
    if_exists='replace',    # Reemplazar si la tabla ya existe
    index=False             # No guardar el índice de Pandas
)

# 3. Consulta SQL Estratégica (Feature Engineering)
sql_query = """
SELECT
    Id,
    ProductId,
    Score,
    Compound_Score,
    -- CREACIÓN DE LA CARACTERÍSTICA CLAVE DE NEGOCIO: Sentiment_Category
    CASE
        -- Sentimiento Positivo (Promotor): Compound Score >= 0.05
        WHEN Compound_Score >= 0.05 THEN 'Positivo'
        -- Sentimiento Negativo (Detractor): Compound Score <= -0.05
        WHEN Compound_Score <= -0.05 THEN 'Negativo'
        -- Sentimiento Neutro (Pasivo): El resto
        ELSE 'Neutro'
    END AS Sentiment_Category
FROM
    reviews_sentiment
"""

# Ejecutar la consulta SQL y cargar los resultados en un nuevo DataFrame para BI
df_final_bi = pd.read_sql_query(sql_query, conn)

# 4. Cerrar la conexión
conn.close()

print("--- Feature Engineering SQL Finalizado con Éxito ---")
print("Se creó la columna 'Sentiment_Category' para BI:")
print(df_final_bi.head())
print(f"\nDistribución de Sentimiento:\n{df_final_bi['Sentiment_Category'].value_counts()}")

--- Feature Engineering SQL Finalizado con Éxito ---
Se creó la columna 'Sentiment_Category' para BI:
   Id ProductId  Score  Compound_Score Sentiment_Category
0   1      B001      5          0.8402           Positivo
1   2      B008      1         -0.7184           Negativo
2   3      B006      5          0.6808           Positivo
3   4      B007      2         -0.3089           Negativo
4   5      B007      4          0.8720           Positivo

Distribución de Sentimiento:
Sentiment_Category
Positivo    4
Negativo    3
Name: count, dtype: int64


In [6]:
# CELDAS DE EXPORTACIÓN A CSV PARA USO EN BI/GITHUB

import os # Importamos la librería 'os' para interactuar con el sistema de archivos (opcional, pero útil)

# 1. Define el nombre del archivo de salida
output_file_name = 'reviews_sentiment_TRANSFORMADO.csv'

# 2. Exportar el DataFrame a CSV
# index=False es crucial para evitar agregar una columna de índice innecesaria
df_final_bi.to_csv(output_file_name, index=False)

# 3. Confirmación (Usamos print para evitar errores de sintaxis)
print(f"--- Exportación Finalizada ---")
print(f"El archivo '{output_file_name}' ha sido creado en el directorio actual de Colab.")
print(f"Ruta completa: {os.getcwd()}/{output_file_name}")

--- Exportación Finalizada ---
El archivo 'reviews_sentiment_TRANSFORMADO.csv' ha sido creado en el directorio actual de Colab.
Ruta completa: /content/reviews_sentiment_TRANSFORMADO.csv
